# Relational Database

## a. Import Library

In [19]:
import mysql.connector
import pandas as pd

## b. Connect to database

In [118]:
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='db_test_movie'
)

## c. Create Table

Tabel Sutradara

In [104]:
# Create Table 
cursor = connection.cursor()
try:
    query = """
    CREATE OR REPLACE TABLE sutradara(
        Directors_id INT AUTO_INCREMENT PRIMARY KEY,
        Name_directors VARCHAR(300)
    )
    """
    cursor.execute(query)
    connection.commit()
    print("Query executed successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Query executed successfully.


Tabel Film

In [105]:
# Create Table 
cursor = connection.cursor()
try:
    query = """
    CREATE OR REPLACE TABLE film(
        Movie_id INT AUTO_INCREMENT PRIMARY KEY,
        Directors_id INT,
        Title VARCHAR(255),
        Rating FLOAT,
        Year INT(4),
        Month VARCHAR(20),
        Certificate VARCHAR(10),
        Runtime INT(4),
        Genre VARCHAR(50),
        Filming_location VARCHAR(300),
        Budget INT,
        Income INT,
        Country_of_origin VARCHAR(255),
        
        FOREIGN KEY(Directors_id) REFERENCES sutradara(Directors_id)
    )
    """
    cursor.execute(query)
    connection.commit()
    print("Query executed successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Query executed successfully.


Tabel Pemeran Film

In [106]:
# Create Table 
cursor = connection.cursor()
try:
    query = """
    CREATE OR REPLACE TABLE pemeran_film(
        Stars_id INT AUTO_INCREMENT PRIMARY KEY,
        Name_stars VARCHAR(300)
    )
    """
    cursor.execute(query)
    connection.commit()
    print("Query executed successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Query executed successfully.


Tabel Detail Pemeran Film

In [159]:
# Create Table 
cursor = connection.cursor()
try:
    query = """
    CREATE OR REPLACE TABLE detail_cast_film(
        Detail_id INT AUTO_INCREMENT PRIMARY KEY,
        Stars_id INT,
        Movie_id INT,
        
        CONSTRAINT fk_pemeran
            FOREIGN KEY(Stars_id) REFERENCES pemeran_film(Stars_id),
        CONSTRAINT fk_film
            FOREIGN KEY(Movie_id) REFERENCES film(Movie_id)
        
    )
    """
    cursor.execute(query)
    connection.commit()
    cursor.close()
    print("Query executed successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Query executed successfully.


## d. Insert Data into Table

create select function

In [115]:
def select(db, table, columns, condition):
    try:
        if db.is_connected():
            cursor = db.cursor()
            query = f"SELECT {columns} FROM {table} {condition}"
            cursor.execute(query)
            result = cursor.fetchall()
            column_names = [column[0] for column in cursor.description]
            cursor.close()
            return result, column_names
    except mysql.connector.Error as err:
        print(f"Error: {err}")

create function insert

In [161]:
def insert(db, table, columns, data):
    try:
        if db.is_connected():
            if(table == "detail_cast_film"):
                cursor = db.cursor()
                insert_query = f"INSERT INTO {table} ({columns}) VALUES ({'%s, %s'})"
                cursor.executemany(insert_query, data)
                db.commit()
                cursor.close()
                print("Insert Data Success")
            elif isinstance(data, list):
                cursor = db.cursor()
                insert_query = f"INSERT INTO {table} ({columns}) VALUES ({'%s'})"
                values_list = [(item,) for item in data]
                cursor.executemany(insert_query, values_list)
                db.commit()
                cursor.close()
                print("Insert Data Success")
            else:
                cursor = db.cursor()
                insert_query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"
                values_list = [tuple(row) for row in data.values]
                cursor.executemany(insert_query, values_list)
                db.commit()
                cursor.close()
                print("Insert Data Success")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

In [175]:
def select(db, table, columns, condition):
    try:
        if db.is_connected():
            cursor = db.cursor()
            query = f"SELECT {columns} FROM {table} {condition}"
            cursor.execute(query)
            result = cursor.fetchall()
            column_names = [column[0] for column in cursor.description]
            cursor.close()
            return result, column_names
    except mysql.connector.Error as err:
        print(f"Error: {err}")

Get Data From Movie

In [121]:
try:
    result, column_names = select(connection, "movie", "*", "")
    # convert data from database into dataframe
    data_movie = pd.DataFrame(result, columns=column_names)
    print(data_movie)
except mysql.connector.Error as err:
    print(f"Erro Select: {err}")

      movie_id                                       Title  Rating  Year  \
0            1                    Avatar: The Way of Water     7.8  2022   
1            2              Guillermo del Toro's Pinocchio     7.6  2022   
2            3                                Bullet Train     7.3  2022   
3            4                                       M3gan     6.4  2022   
4            5                                   Amsterdam     6.1  2022   
...        ...                                         ...     ...   ...   
1599      1600                       House of Sand and Fog     7.5  2003   
1600      1601                                  In the Cut     5.4  2003   
1601      1602  Lara Croft Tomb Raider: The Cradle of Life     5.5  2003   
1602      1603                                     Gothika     5.8  2003   
1603      1604                                  Open Water     5.8  2003   

         Month Certificate  Runtime                           Directors  \
0     Decemb

Insert Data to Table sutradara (data master)

In [110]:
try:
    cursor = connection.cursor()
    sutradara_list_data = data_movie[['Directors']]
    sutradara_list_data = sutradara_list_data['Directors'].unique().tolist()
    columns = "Name_directors"
    insert(connection, 'sutradara', columns, sutradara_list_data)
    cursor.close()
except mysql.connector.Error as err:
    print(f"Erro Insert: {err}")

Insert Data Success


Insert Data Into Table Film

In [135]:
try:
    
    table = "movie m"
    column = "s.Directors_id, m.Title, m.Rating, m.Year, m.Month, m.Certificate, m.Runtime, m.Genre, m.Filming_location, m.Budget, m.Income, m.Country_of_origin"
    condition = "JOIN sutradara s ON m.Directors = s.Name_directors"
    # Get Data Movie where same director
    sutradara_data, column_names = select(connection, table, column, condition)
    # Insert Data Film
    sutradara_data = pd.DataFrame(result, columns=column_names)
    insert(connection, 'film', column_names, sutradara_data)
    
except mysql.connector.Error as err:
    print(f"Erro Insert: {err}")

Insert Data Success


Insert Data Pemeran Film

In [144]:
try:
    table = "movie"
    column = "Stars"
    condition = ""
    # Get Data Stars
    stars_data, column_names = select(connection, table, column, condition)
    # make list names of stars
    stars_names_list = [stars for row in stars_data for stars in row[0].split(', ')]
    # Get Unique Data from list names of stars
    unique_star_names = set(stars_names_list)
    unique_star_names = list(unique_star_names)
    # Insert Data Film
    insert(connection, 'pemeran_film', "Name_stars", unique_star_names)
    
except mysql.connector.Error as err:
    print(f"Erro Insert: {err}")

Insert Data Success


Insert Data into Detail Cast Film

In [162]:
try:
    # Retrieve data Stars from the movie table
    table = "movie"
    column = "movie_id, Stars"
    condition = ""
    movie_casts_data, movie_casts_column = select(connection, table, column, condition)
    # Retrieve data from the pemeran film table
    table = "pemeran_film"
    column = "Stars_id, Name_stars"
    condition = ""
    pf_casts_data, pf_casts_column = select(connection, table, column, condition)
    # Prepare a dictionary to map cast names to cast_id values
    cast_name_to_id = {cast_name: cast_id for cast_id, cast_name in pf_casts_data}
    # Insert Data to Film Cast Detail Table
    table = "detail_cast_film"
    column = "Stars_id, Movie_id"
    data_to_insert = [(cast_name_to_id[cast_name], movie_id) for movie_id, casts in movie_casts_data for cast_name in casts.split(', ')]
    insert(connection, table, column, data_to_insert)
    
except mysql.connector.Error as err:
    print(f"Erro Insert: {err}")

Insert Data Success


## e. Performing Analytic Data

i. Grouping data tersebut berdasarkan sertifikasi film

In [163]:
try:
    table = "film"
    columns = "Certificate, COUNT(*) as jumlah_film"
    condition = "GROUP BY Certificate"
    result, column_names = select(connection, table, columns, condition)
    # convert data from database into dataframe
    grouping_certificate_data = pd.DataFrame(result, columns=column_names)
    print(grouping_certificate_data)
except mysql.connector.Error as err:
    print(f"Erro Select: {err}")

  Certificate  jumlah_film
0           G           18
1       NC-17            4
2   Not Rated           28
3          PG          165
4       PG-13          656
5           R          724
6       TV-14            1
7       TV-MA            7
8     Unrated            1


ii. Tampilkan sutradara yang telah menyutradarai lebih dari 1 film dan film-film tersebut
memiliki rating diatas 7.5

In [164]:
try:
    table = "film f"
    columns = "s.Name_directors"
    condition = "JOIN sutradara s ON f.Directors_id = s.Directors_id WHERE f.Rating > 7.5 GROUP BY s.Name_directors HAVING COUNT(*) > 1"
    result, column_names = select(connection, table, columns, condition)
    # convert data from database into dataframe
    directors_data = pd.DataFrame(result, columns=column_names)
    print(directors_data)
except mysql.connector.Error as err:
    print(f"Erro Select: {err}")

              Name_directors
0        Alejandro G I rritu
1             Alfonso Cuar n
2                    Ang Lee
3   Anthony Russo, Joe Russo
4                Ben Affleck
5               Bong Joon Ho
6                  Brad Bird
7               Bryan Singer
8          Christopher Nolan
9             Clint Eastwood
10           Damien Chazelle
11          Darren Aronofsky
12             David Fincher
13           David O Russell
14               David Yates
15          Denis Villeneuve
16              Edgar Wright
17              Edward Zwick
18     Ethan Coen, Joel Coen
19          Gabriele Muccino
20               Guy Ritchie
21                J J Abrams
22             James Cameron
23                James Gunn
24             James Mangold
25                Joe Wright
26           Joseph Kosinski
27               Joss Whedon
28              Marc Forster
29                 Marc Webb
30           Martin McDonagh
31           Martin Scorsese
32               Matt Reeves
33            

iii. Tampilkan film apa saja yang meraih keuntungan dalam produksi nya

In [165]:
try:
    table = "film"
    columns = "Title"
    condition = "WHERE Income > Budget"
    result, column_names = select(connection, table, columns, condition)
    # convert data from database into dataframe
    directors_data = pd.DataFrame(result, columns=column_names)
    print(directors_data)
except mysql.connector.Error as err:
    print(f"Erro Select: {err}")

                                           Title
0                       Avatar: The Way of Water
1                                   Bullet Train
2                                          M3gan
3                                  Violent Night
4                                      The Whale
...                                          ...
1347                       House of Sand and Fog
1348                                  In the Cut
1349  Lara Croft Tomb Raider: The Cradle of Life
1350                                     Gothika
1351                                  Open Water

[1352 rows x 1 columns]


iv. Bulan mana yang memiliki jumlah film yang diproduksi terbanyak

In [166]:
try:
    table = "film"
    columns = "Month"
    condition = "GROUP BY Month ORDER BY COUNT(*) DESC LIMIT 1"
    result, column_names = select(connection, table, columns, condition)
    # convert data from database into dataframe
    directors_data = pd.DataFrame(result, columns=column_names)
    print(directors_data)
except mysql.connector.Error as err:
    print(f"Erro Select: {err}")

      Month
0  November


v. Sebutkan 5 film apa saja yang memiliki rating tertinggi untuk perbulannya (urutkan dari
rating yang terkecil ke yang terbesar)

In [176]:
try:
    table = "( SELECT *, ROW_NUMBER() OVER (PARTITION BY Month ORDER BY Rating DESC) AS rn FROM film ) ranked "
    columns = "Title, Month, Rating"
    condition = "WHERE rn <= 5 GROUP BY Month ORDER BY Rating LIMIT 5"
    # print(type(table))
    result, column_names = select(connection, table, columns, condition)
    # convert data from database into dataframe
    directors_data = pd.DataFrame(result, columns=column_names)
    print(directors_data)
except mysql.connector.Error as err:
    print(f"Erro Select: {err}")

                  Title     Month  Rating
0        Shutter Island  February     8.2
1  Inglourious Basterds    August     8.3
2              The Hunt   January     8.3
3               WALLÂ·E      June     8.4
4   The Lives of Others     March     8.4
